In [1]:
from ambiance import Atmosphere
import numpy as np

from global_variables.solver import EquationSystem
from global_variables.registry import VariableRegistry, Variable

registry = VariableRegistry("aero_vars.yaml")

In [6]:
system = EquationSystem(registry, targets={'R','W_f'})
R_solver = system.create_solver()

def velocity(M,h):
  atmo = Atmosphere(h * 0.3048)
  return atmo.speed_of_sound[-1] * M / 0.3048
def rho_func(h):
  atmo = Atmosphere(h * 0.3048)
  return atmo.density[-1] * 0.00194032

AR = 8
b = 300
S_ref = b**2 / AR
print(S_ref)
args = {
    "C_D0":0.017,
    "S_":S_ref,
    "TSFC":.18/3600,
    "V":velocity(M=.87,h=3.7e4),
    #"W_max":2e6,
    "W_pax":205,
    "We_Wmax":.5,
    "b":b,
    "e":0.9,
    "n_pax":1254,
    "rho_h":rho_func(3.7e4)
}

from scipy.optimize import minimize
def find_max_weight(desired_range,max_weight=2e6,**kwargs):
    return minimize(
        lambda w_max: (R_solver['R'](W_max=w_max,**kwargs) - desired_range)**2,
        x0=max_weight,
        #method="Nelder-Mead"
    )
out = find_max_weight(1e4*6076.12,max_weight=1.75e6,**args)
print(f"Max Weight: {out['x'][0]}")
fuel_weight = R_solver['W_f'](W_max=out['x'],**args)[0]
print(f"Fuel Weight: {fuel_weight}")
fuel_volume = fuel_weight / 4.432
print(f"Fuel Volume: {fuel_volume}")

11250.0
Max Weight: 927564.5978951582
Fuel Weight: 206712.2989475791
Fuel Volume: 46640.86167589781


In [ ]:
from aircraft_design.core.base import Component, Position


class WingBody(Component):
    """Wing-body aircraft configuration"""
    def __init__(self, name: str):
        super().__init__(name)
        self.configuration: Dict[str, Any] = {}
    
    
